In [4]:
class TaintTrackerEntry:
    def __init__(self, register_id):
        self.register_id = register_id  # 8-bit 寄存器ID
        self.tainted = 0  # 1-bit 是否被污染
        self.srf_id = 0  # 8-bit SRF寄存器ID
        self.offset = 0  # 8-bit 偏移
        self.mapped = 0  # 1-bit 是否被映射

    def __repr__(self):
        return (f"RegID: {self.register_id}, Tainted: {self.tainted}, SRF_ID: {self.srf_id}, "
                f"Offset: {self.offset}, Mapped: {self.mapped}")


class TaintTracker:
    def __init__(self, max_entries=16):
        self.entries = {}  # 存储所有的寄存器条目，按寄存器ID索引
        self.max_entries = max_entries  # 最大条目数
        self.access_order = []  # 记录访问顺序，用于LRU策略

    def _update_access_order(self, register_id):
        """ 更新访问顺序，移到最后表示最近使用 """
        if register_id in self.access_order:
            self.access_order.remove(register_id)
        self.access_order.append(register_id)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_register = self.access_order.pop(0)
            del self.entries[oldest_register]
            print(f"Evicted Register {oldest_register} due to capacity limit.")

    def taint_register(self, register_id, srf_id, offset):
        """ 将指定寄存器标记为污染状态，并映射到SRF """
        if register_id not in self.entries:
            # 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            # 创建新条目
            self.entries[register_id] = TaintTrackerEntry(register_id)
        
        entry = self.entries[register_id]
        entry.tainted = 1  # 标记寄存器被污染
        entry.srf_id = srf_id  # 映射到的SRF ID
        entry.offset = offset  # 设置偏移
        entry.mapped = 1  # 设置映射标记

        # 更新访问顺序
        self._update_access_order(register_id)

    def clear_taint(self, register_id):
        """ 清除指定寄存器的污染状态 """
        if register_id in self.entries:
            entry = self.entries[register_id]
            entry.tainted = 0
            entry.mapped = 0  # 清除映射标记

            # 更新访问顺序（清除污染状态后仍视为最近访问）
            self._update_access_order(register_id)

    def is_tainted(self, register_id):
        """ 检查指定寄存器是否被污染 """
        if register_id in self.entries:
            return self.entries[register_id].tainted == 1
        return False

    def get_entry(self, register_id):
        """ 返回指定寄存器的条目 """
        if register_id in self.entries:
            return self.entries[register_id]
        return None

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Taint Tracker Entries (Max Entries: {self.max_entries}):\n{entries_repr}"



class StrideDetectorEntry:
    def __init__(self, pc):
        self.pc = pc  # 48-bit 程序计数器
        self.previous_address = 0  # 48-bit 前一个地址
        self.stride = 0  # 8-bit 步长
        self.last_prefetch = 0  # 48-bit 上次预取地址
        self.seen = 0  # 1-bit 是否检测到
        self.lil = 0  # 16-bit 最后一个间接加载
        self.stride_confidence = 0  # 2-bit 步长置信度
        self.lil_confidence = 0  # 2-bit LIL置信度

    def __repr__(self):
        return (f"PC: {hex(self.pc)}, PrevAddr: {self.previous_address}, Stride: {self.stride}, "
                f"LastPrefetch: {self.last_prefetch}, Seen: {self.seen}, LIL: {self.lil}, "
                f"StrideConf: {self.stride_confidence}, LILConf: {self.lil_confidence}")


class StrideDetector:
    def __init__(self, max_entries=16):
        self.entries = {}  # 存储步长检测的条目
        self.max_entries = max_entries  # 最大条目数
        self.access_order = []  # 记录访问顺序，用于LRU替换策略

    def _update_access_order(self, pc):
        """ 更新访问顺序，移到最后表示最近使用 """
        if pc in self.access_order:
            self.access_order.remove(pc)
        self.access_order.append(pc)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_pc = self.access_order.pop(0)  # 移除最久未使用的条目
            del self.entries[oldest_pc]
            print(f"Evicted PC {hex(oldest_pc)} due to capacity limit.")

    def detect_stride(self, pc, current_address):
        """ 检测步长模式，并更新步长置信度 """
        # 如果没有该PC的记录，则初始化一个条目
        if pc not in self.entries:
            # 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            # 创建新的Stride Detector条目
            self.entries[pc] = StrideDetectorEntry(pc)
        
        entry = self.entries[pc]
        stride_detected = False

        # 计算当前步长
        stride = current_address - entry.previous_address

        # 更新步长置信度和步长值
        if stride == entry.stride:
            # 如果当前步长与之前相同，则提高置信度
            entry.stride_confidence = min(entry.stride_confidence + 1, 3)
            stride_detected = True
        else:
            # 否则更新步长，并重置置信度
            entry.stride = stride
            entry.stride_confidence = 1
        
        # 更新条目中的前一个地址
        entry.previous_address = current_address

        # 更新访问顺序
        self._update_access_order(pc)

        return stride_detected, entry.stride_confidence

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Stride Detector Entries (Max Entries: {self.max_entries}):\n{entries_repr}"


class LoopBoundEntry:
    def __init__(self, pc):
        self.pc = pc  # 48-bit 程序计数器
        self.comp_pc = 0  # 48-bit 比较指令的PC值
        self.ewma = 0  # 9-bit 指数加权移动平均
        self.iteration_counter = 0  # 9-bit 迭代计数器
        self.loop_increment = 0  # 16-bit 循环增量
        self.confidence = 0  # 2-bit 置信度
        self.source_val_a = 0  # 64-bit 源值A
        self.source_val_b = 0  # 64-bit 源值B
        self.reg_id_a = 0  # 5-bit 寄存器ID A
        self.reg_id_b = 0  # 5-bit 寄存器ID B

    def __repr__(self):
        return (f"PC: {hex(self.pc)}, Comp-PC: {self.comp_pc}, EWMA: {self.ewma}, "
                f"Iteration: {self.iteration_counter}, Increment: {self.loop_increment}, "
                f"Confidence: {self.confidence}, SrcA: {self.source_val_a}, SrcB: {self.source_val_b}, "
                f"RegID_A: {self.reg_id_a}, RegID_B: {self.reg_id_b}")


class LoopBoundDetector:
    def __init__(self, max_entries=16):
        self.entries = {}  # 存储所有的循环条目，按PC索引
        self.max_entries = max_entries  # 最大条目数
        self.access_order = []  # 记录访问顺序，用于LRU替换策略

    def _update_access_order(self, pc):
        """ 更新访问顺序，移到最后表示最近使用 """
        if pc in self.access_order:
            self.access_order.remove(pc)
        self.access_order.append(pc)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_pc = self.access_order.pop(0)  # 移除最久未使用的条目
            del self.entries[oldest_pc]
            print(f"Evicted Loop Entry for PC {oldest_pc} due to capacity limit.")

    def update_loop_bound(self, pc, comp_pc, ewma, iteration, increment, src_a, src_b, reg_id_a, reg_id_b):
        """ 更新指定PC的循环边界信息，增加寄存器ID """
        if pc not in self.entries:
            # 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            # 创建新的Loop Bound条目
            self.entries[pc] = LoopBoundEntry(pc)
        
        entry = self.entries[pc]
        entry.comp_pc = comp_pc
        entry.ewma = ewma
        entry.iteration_counter = iteration
        entry.loop_increment = increment
        entry.source_val_a = src_a
        entry.source_val_b = src_b
        entry.reg_id_a = reg_id_a  # 设置寄存器ID A
        entry.reg_id_b = reg_id_b  # 设置寄存器ID B

        # 根据迭代次数和EWMA更新置信度
        entry.confidence = min(3, entry.confidence + 1 if iteration <= ewma else entry.confidence - 1)

        # 更新访问顺序
        self._update_access_order(pc)

    def clear_entry(self, pc):
        """ 清除指定PC的循环条目 """
        if pc in self.entries:
            del self.entries[pc]
            print(f"Cleared Loop Entry for PC {hex(pc)}")
            if pc in self.access_order:
                self.access_order.remove(pc)

    def get_entry(self, pc):
        """ 返回指定PC的条目 """
        if pc in self.entries:
            return self.entries[pc]
        return None

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Loop Bound Detector Entries (Max Entries: {self.max_entries}):\n{entries_repr}"

class HSLR:
    def __init__(self):
        self.pc = 0  # 48-bit 程序计数器
        self.mask = 0  # 16-bit 控制流掩码

    def write(self, pc, mask):
        """ 更新HSLR寄存器的内容 """
        self.pc = pc
        self.mask = mask

    def __repr__(self):
        return f"HSLR Register: PC: {hex(self.pc)}, Mask: {self.mask}"

import math

class ScoreboardEntry:
    def __init__(self, pc, status=0, return_counter=0, max_scalars=16):
        self.pc = pc  # 48-bit 程序计数器
        self.status = status  # 5-bit 状态标志位，表示指令状态
        self.return_counter = return_counter  # 返回计数器，初始为 0
        self.max_scalars = max_scalars  # 最大标量指令数，用于设置计数器的最大值

        # 计算 log2(N + 1) 的比特数，表示返回计数器的比特位宽
        self.return_counter_bits = math.ceil(math.log2(max_scalars + 1))

    def __repr__(self):
        return (f"PC: {hex(self.pc)}, Status: {self.status}, "
                f"Return Counter: {self.return_counter} / {self.max_scalars} (Bits: {self.return_counter_bits})")

    def update_status(self, new_status):
        """ 更新指令的状态 """
        self.status = new_status

    def increment_return_counter(self, increment=1):
        """ 增加返回计数器值，表示发出了更多的标量指令 """
        self.return_counter = min(self.max_scalars, self.return_counter + increment)

    def decrement_return_counter(self, decrement=1):
        """ 更新返回计数器，指令执行完毕时递减计数器 """
        self.return_counter = max(0, self.return_counter - decrement)  # 确保计数器不为负值

    def is_completed(self):
        """ 判断所有标量指令是否已经执行完毕 """
        return self.return_counter == 0


class Scoreboard:
    def __init__(self, max_entries=32, max_scalars=16):
        self.entries = {}  # 存储所有的Scoreboard条目，按PC索引
        self.max_entries = max_entries  # 最大条目数
        self.max_scalars = max_scalars  # 每个条目中标量指令的最大数量
        self.access_order = []  # 记录访问顺序，用于LRU替换策略

    def _update_access_order(self, pc):
        """ 更新访问顺序，移到最后表示最近使用 """
        if pc in self.access_order:
            self.access_order.remove(pc)
        self.access_order.append(pc)

    def _evict_entry(self):
        """ 使用LRU策略移除最早访问的条目 """
        if len(self.access_order) > 0:
            oldest_pc = self.access_order.pop(0)  # 移除最久未使用的条目
            del self.entries[oldest_pc]
            print(f"Evicted Scoreboard Entry for PC {oldest_pc} due to capacity limit.")

    def add_entry(self, pc, status=0):
        """ 添加新的Scoreboard条目，并初始化返回计数器 """
        if pc not in self.entries:
            # 如果当前条目数超过最大限制，执行替换策略
            if len(self.entries) >= self.max_entries:
                self._evict_entry()
            
            # 创建新的 Scoreboard 条目，并将返回计数器初始化为标量指令数量
            self.entries[pc] = ScoreboardEntry(pc, status, return_counter=self.max_scalars, max_scalars=self.max_scalars)
            print(f"Added Scoreboard Entry for PC {hex(pc)} with Return Counter set to {self.max_scalars}")

        # 更新访问顺序
        self._update_access_order(pc)

    def update_entry_status(self, pc, new_status):
        """ 更新指定PC的状态 """
        if pc in self.entries:
            self.entries[pc].update_status(new_status)
            print(f"Updated Status for PC {hex(pc)} to {new_status}")

    def decrement_return_counter(self, pc, decrement=1):
        """ 更新指定PC的返回计数器 """
        if pc in self.entries:
            self.entries[pc].decrement_return_counter(decrement)
            print(f"Updated Return Counter for PC {hex(pc)} by decrementing {decrement}")
            # 检查返回计数器是否已归零
            if self.entries[pc].is_completed():
                print(f"All scalar instructions for PC {hex(pc)} have completed.")

    def get_entry(self, pc):
        """ 返回指定PC的条目 """
        if pc in self.entries:
            return self.entries[pc]
        return None

    def clear_entry(self, pc):
        """ 清除指定PC的Scoreboard条目 """
        if pc in self.entries:
            del self.entries[pc]
            print(f"Cleared Scoreboard Entry for PC {hex(pc)}")
            if pc in self.access_order:
                self.access_order.remove(pc)

    def __repr__(self):
        entries_repr = "\n".join([str(entry) for entry in self.entries.values()])
        return f"Scoreboard Entries (Max Entries: {self.max_entries}):\n{entries_repr}"
    
  


class ScalarVectorUnit:
    def __init__(self, scalar_vector_length=16):
        # Scalar Vector Unit 的配置项
        self.scalar_vector_length = scalar_vector_length  # 定义 SVU 中标量拷贝的数量
        self.srf = {}  # Speculative Register File (SRF)，存储标量拷贝的执行结果
        self.taunt_tracker = {}  # 污染追踪器，用于标记被污染的寄存器
        
    def generate_svi(self, pc, base_address, stride):
        """ 根据基地址和步长生成 Scalar Vector Instructions (SVIs) """
        svi_list = []
        for i in range(self.scalar_vector_length):
            # 每个 SVI 是一个标量指令的拷贝，地址为 base_address + i * stride
            new_address = base_address + i * stride
            svi_list.append((pc, new_address))
        return svi_list

    def execute_svi(self, svi_list):
        """ 模拟执行生成的 SVIs，并将结果写入推测寄存器文件 (SRF) """
        for (pc, addr) in svi_list:
            # 模拟加载数据：将地址值存入 SRF 中，表示已加载该地址
            self.srf[pc] = addr
            print(f"Executed SVI for PC: {hex(pc)} at address: {addr}")

    def update_taint_tracker(self, reg_id):
        """ 更新污染追踪器，标记某个寄存器已经被污染 """
        self.taunt_tracker[reg_id] = True
        print(f"Register {reg_id} marked as tainted.")

    def clear_taint(self, reg_id):
        """ 清除某个寄存器的污染标记 """
        if reg_id in self.taunt_tracker:
            del self.taunt_tracker[reg_id]
            print(f"Taint cleared for Register {reg_id}.")

    def __repr__(self):
        srf_repr = "\n".join([f"PC: {hex(pc)}, Data: {data}" for pc, data in self.srf.items()])
        return f"Scalar-Vector Unit (SVU) State:\nSRF Entries:\n{srf_repr}\nTaint Tracker: {self.taunt_tracker}"


In [5]:
class ScalarVectorRunahead:
    def __init__(self, max_entries=32, scalar_vector_length=16):
        # 初始化各个模块
        self.stride_detector = StrideDetector(max_entries=max_entries)  # 步长检测器
        self.taint_tracker = TaintTracker(max_entries=max_entries)  # 污染追踪器
        self.hslr = HSLR()  # 头步长加载寄存器
        self.svu = ScalarVectorUnit(scalar_vector_length)  # 标量向量单元
        self.scoreboard = Scoreboard(max_entries=max_entries, max_scalars=scalar_vector_length)  # Scoreboard

    def run_svr(self, instructions):
        """
        运行 SVR 系统，根据给定的指令流进行标量向量化操作。
        :param instructions: List[Dict] 指令流，每个指令包含 PC、地址、寄存器等信息
        """
        for instruction in instructions:
            pc = instruction['pc']
            address = instruction['address']
            reg = instruction['destination_register']
            
            # 1. 步长检测 (SD)
            stride_detected, stride_confidence = self.stride_detector.detect_stride(pc, address)
            if stride_detected:
                # 检测到步长模式，更新HSLR，并生成标量向量指令（SVI）
                self.hslr.write(pc, mask=0xFFFF)  # 更新 HSLR 寄存器
                svi_list = self.svu.generate_svi(pc, base_address=address, stride=self.stride_detector.entries[pc].stride)
                print(f"Stride detected for PC {hex(pc)}: Generating SVIs {svi_list}")

                # 2. 标量向量单元 (SVU) 执行生成的 SVI
                self.svu.execute_svi(svi_list)

                # 3. 更新 Scoreboard
                self.scoreboard.add_entry(pc, status=1)  # 标记该步长指令已进入 SVR 模式
                for _ in svi_list:
                    self.scoreboard.decrement_return_counter(pc)  # 逐个减少返回计数器

            # 4. 污染追踪 (TT)
            if self.taint_tracker.is_tainted(reg):
                print(f"Register {reg} is tainted and part of an indirect chain. Updating dependencies.")
            else:
                # 将寄存器标记为污染状态
                self.taint_tracker.taint_register(register_id=reg, srf_id=1, offset=0)

            # 5. 检查指令是否已经完成执行
            if self.scoreboard.get_entry(pc) and self.scoreboard.get_entry(pc).is_completed():
                print(f"All scalar instructions for PC {hex(pc)} have completed. Clearing Scoreboard entry.")
                self.scoreboard.clear_entry(pc)

    def __repr__(self):
        return (f"Scalar Vector Runahead State:\n"
                f"Stride Detector: {self.stride_detector}\n"
                f"Taint Tracker: {self.taint_tracker}\n"
                f"HSLR: {self.hslr}\n"
                f"SVU: {self.svu}\n"
                f"Scoreboard: {self.scoreboard}")


if __name__ == '__main__':
    svr = ScalarVectorRunahead(max_entries=4, scalar_vector_length=4)

    # 假设简易指令流，每个指令包含PC、内存地址、目标寄存器等信息
    # for 循环内相同pc访问不同地址
    # 这一步拿到的信息就已经是csr转换后离散的了
    instructions = [
        {'pc': 0x1ABC, 'address': 1000, 'destination_register': 1},
        {'pc': 0x1ABC, 'address': 1016, 'destination_register': 2},
        {'pc': 0x1ABC, 'address': 1032, 'destination_register': 3},
        {'pc': 0x1ABC, 'address': 1048, 'destination_register': 4},
        {'pc': 0x2DEF, 'address': 2000, 'destination_register': 5},
        {'pc': 0x2DEF, 'address': 2016, 'destination_register': 6},
    ]

    # 运行 SVR 系统
    svr.run_svr(instructions)

    # 打印 SVR 系统的最终状态
    print(svr)




Stride detected for PC 0x1abc: Generating SVIs [(6844, 1032), (6844, 1048), (6844, 1064), (6844, 1080)]
Executed SVI for PC: 0x1abc at address: 1032
Executed SVI for PC: 0x1abc at address: 1048
Executed SVI for PC: 0x1abc at address: 1064
Executed SVI for PC: 0x1abc at address: 1080
Added Scoreboard Entry for PC 0x1abc with Return Counter set to 4
Updated Return Counter for PC 0x1abc by decrementing 1
Updated Return Counter for PC 0x1abc by decrementing 1
Updated Return Counter for PC 0x1abc by decrementing 1
Updated Return Counter for PC 0x1abc by decrementing 1
All scalar instructions for PC 0x1abc have completed.
All scalar instructions for PC 0x1abc have completed. Clearing Scoreboard entry.
Cleared Scoreboard Entry for PC 0x1abc
Stride detected for PC 0x1abc: Generating SVIs [(6844, 1048), (6844, 1064), (6844, 1080), (6844, 1096)]
Executed SVI for PC: 0x1abc at address: 1048
Executed SVI for PC: 0x1abc at address: 1064
Executed SVI for PC: 0x1abc at address: 1080
Executed SVI for 